In [ ]:
!pip install bert-embedding 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 7.3 MB/s 
     |████████████████████████████████| 29.6 MB 52.9 MB/s 
     |████████████████████████████████| 13.8 MB 36.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-py3-none-any.whl size=259930 sha256=a608445b10d98c320da4456d8a3c1ede27a195da0b47a9590ec78bd3ba725951
  Stored in directory: /root/.cache/pip/wheels/a6/41/8f/45bd1c58055d87aee5a71b6756a427ea8d92e506b3a9d17370
Successfully built gluonnlp
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

In [ ]:
import re
import pandas as pd
import numpy as np
from bert_embedding import BertEmbedding
from tqdm import tqdm
import pickle


In [ ]:
def extract_emb(path):
  dataset = pd.read_csv(path)
  #Embedding
  bert_embedding = BertEmbedding(model='bert_24_1024_16', dataset_name='book_corpus_wiki_en_cased')
  poems_emb = []
  poems_tokens = []
  for poem in tqdm(dataset["Poem"]): 
    #Remove all the line breaks
    clearedPoem = re.sub('\n{2,}', "\n", poem)
    #Split by line breaks
    sentences = clearedPoem.split('\n')
    #Embedding
    result = bert_embedding(sentences)
    concatenated_emb = np.concatenate([w[None, ...] for sentence in result for w in sentence[1]]).T
    poems_emb.append(concatenated_emb)
    poems_tokens.append([w for sentence in result for w in sentence[0]])
  
  return poems_emb, poems_tokens

In [ ]:
embddings, tokens = extract_emb("./PoetryData.csv")

100%|██████████| 300/300 [1:25:34<00:00, 17.12s/it]


In [ ]:
with open('bert.npy', 'wb') as f:
  for poem in embddings:
    np.save(f, poem)

In [ ]:
with open("tokens.p", "wb" ) as f:
  pickle.dump(tokens,  f)

In [ ]:
fake_embddings, fake_tokens = extract_emb("./PoetryData_fake.csv")

Vocab file is not found. Downloading.


100%|██████████| 100/100 [15:28<00:00,  9.29s/it]


In [ ]:
with open('fake_bert.npy', 'wb') as f:
  for poem in fake_embddings:
    np.save(f, poem)

In [ ]:
with open("fake_tokens.p", "wb" ) as f:
  pickle.dump(fake_tokens,  f)